## Question 3 : Multiresolution Blending and Feathering
In this question we blend two given images using Laplacian stack and feathering.
Algorithm is simple: first we create Gaussian stack for each image, then create
Laplacian stack using Gaussian stack. We also create a stack of masks for blending.
Then we blend the images in each level of the stack using their corresponding mask to get blended stack. Finally,
We sum up all images in blended stack to get the final result.
Functions used in the problem are implemented in `q3_funcs.py` and the main code of the problem is
in `q3.py`

### q3_funcs
First function simply uses GaussianBlur method in opencv to blur a given image :

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.signal as signal


def blur_img(img, kernel_size, sigma):
    result = cv2.GaussianBlur(img, (kernel_size, kernel_size), sigma)
    return result

Next two function generate Gaussian and Laplacian stack for a given image :


In [ ]:
def generate_gaussian_stack(img, kernel_size, sigma, stack_size):
    result = []
    result.append(img)
    temp_img = img.copy()
    for i in range(1, stack_size):
        temp_img = blur_img(temp_img, kernel_size, sigma)
        # print("blurred once")
        result.append(temp_img)
    return result

def generate_laplacian_stack(img, kernel_size, sigma, stack_size):
    gaussian_pyramid = generate_gaussian_stack(img, kernel_size, sigma, stack_size)
    result = []
    for i in range(0, stack_size - 1):
        temp = gaussian_pyramid[i].copy() - gaussian_pyramid[i + 1].copy()
        result.append(temp)
    result.append(gaussian_pyramid[-1].copy())
    return result



Function `generate_mask` creates a stack of masks. Each element of the stack is the result of
blurring previous element, so in higher levels, we will have more intensive feathering

In [ ]:
def generate_masks(mask, kernel_size, sigma, size):
    result = []
    new_mask = blur_img(mask, kernel_size, sigma)
    result.append(new_mask)
    for i in range(size):
        new_mask = blur_img(new_mask, kernel_size, sigma)
        # plt.imshow(new_mask)
        # plt.show()
        result.append(new_mask)

    return result



Next function creates a stack of blended images, in the way described
in the begining :

In [ ]:
def generate_blended_stack(img1, img2, masks, kernel_size, sigma, stack_size):
    result = []
    laplacian_stack1 = generate_laplacian_stack(img1, kernel_size, sigma, stack_size)
    laplacian_stack2 = generate_laplacian_stack(img2, kernel_size, sigma, stack_size)
    for i in range(stack_size):
        blended_img = laplacian_stack1[i] * masks[i] + laplacian_stack2[i] * (1 - masks[i])
        result.append(blended_img)
    return result

Final function `blend_images`, takes two images, a stack of masks, a kernel size and sigma for blurring and stack size
as input, and outputs the resulting blended image:

In [ ]:
def blend_images(img1, img2, masks, kernel_size, sigma, stack_size):
    blended_stack = generate_blended_stack(img1, img2, masks, kernel_size, sigma, stack_size)
    result = blended_stack[0].copy()
    for i in range(1, stack_size):
        result = result + blended_stack[i]
    return result


Function `convert_from_float32_to_uint8` is used to scale an image
from interval [0,1] to interval [0,255], with integer valued pixels :

In [ ]:
def convert_from_float32_to_uint8(img):
    max_index, min_index = np.max(img), np.min(img)
    a = 255 / (max_index - min_index)
    b = 255 - a * max_index
    result = (a * img + b).astype(np.uint8)
    return result

### q3.py
In the main file, after reading images, we scale them to the range [0,1], apply above functions on them and rescaling the result
back to range [0,255]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import q3_funcs as funcs

img1 = cv.imread("images/res08.jpg")
img2 = cv.imread("images/res09.jpg")

img_float1 = (img1 / 255).astype('float32')
img_float2 = (img2 / 255).astype('float32')

kernel_size, sigma, n = 151, 30, 10
img_list = funcs.generate_gaussian_stack(img1, kernel_size, sigma, n)
lap_list = funcs.generate_laplacian_stack(img1, kernel_size, sigma, n)

''' creating masks '''
h, w = img1.shape[0], img1.shape[1]
mask = np.zeros(img1.shape, dtype='float32')
mask[:, 0:400] = 1
masks = funcs.generate_masks(mask, 151, 30, n)

result = funcs.blend_images(img_float1, img_float2, masks, kernel_size, sigma, n)
result = funcs.convert_from_float32_to_uint8(result)
cv.imwrite("images/res10.jpg", result)

